<p style="font-weight:bold;"> <span style="font-size: 36px"> Reinsurance</span> </p>

<p style="font-weight:bold;"> <span style="font-size: 21px"> Model</span> </p>

# Data Import

In [0]:
#!eval-notebook "DataImport"
Workspace.InitializeFrom(DataSource);
//ifrs17.Reset(Workspace) //This is not working

In [0]:
ifrs17 = new Ifrs17(Workspace, Scopes, Report);

In [0]:
var reportingYear = 2020; 
var reportingMonth = 12;

# Best Estimate



In [0]:
var pv = ifrs17.PresentValues;
pv.ReportingNode = "CH";
pv.ReportingPeriod = (reportingYear, reportingMonth);
pv.CurrencyType = CurrencyType.Contractual;
pv.ColumnSlices = new string[]{"GroupOfContract", "AmountType"};
pv.DataFilter = null;//new [] {("GroupOfContract", "RP1.1")};
pv.Scenario = null;//"All";
(await pv.ToReportAsync)

# Fulfilment Cash Flow (FCF)

We start by looking at the Fulfilment Cash Flows (FCF). Data and results for both cases are exactly the same.

In [0]:
var fulfillmentCashflows = ifrs17.FulfillmentCashflows;
fulfillmentCashflows.ReportingNode = "CH";
fulfillmentCashflows.ReportingPeriod = (reportingYear, reportingMonth);
fulfillmentCashflows.ColumnSlices = new string[]{"GroupOfContract","EstimateType"};
fulfillmentCashflows.DataFilter = null; //new [] {("GroupOfContract", "RP1.1")};
fulfillmentCashflows.Scenario = "All";
(await fulfillmentCashflows.ToReportAsync) with {Height = 750}

# Written Actuals

Aslo the Actuals for both cases are exactly the same.

In [0]:
var writtenActual = ifrs17.WrittenActuals;
writtenActual.ReportingNode = "CH";
writtenActual.ReportingPeriod = (reportingYear, reportingMonth);
writtenActual.ColumnSlices = new string[]{"GroupOfContract"};
writtenActual.DataFilter =  new [] {("GroupOfContract", "RP1.1")};
(await writtenActual.ToReportAsync) with {Height = 400}

# LRC Technical Margin (TM)  

In [0]:
var technicalMargins = ifrs17.TechnicalMargins;
technicalMargins.ReportingNode = "CH";
technicalMargins.ReportingPeriod = (reportingYear, reportingMonth);
technicalMargins.ColumnSlices = new string[]{"GroupOfContract"};
technicalMargins.DataFilter = null; //new [] {("GroupOfContract", "RP1.1")};
technicalMargins.Scenario = "All";
(await technicalMargins.ToReportAsync) with {Height = 600}

# Contractual Service Margin / Loss Component

In [0]:
var allocatedTechnicalMargins = ifrs17.AllocatedTechnicalMargins;
allocatedTechnicalMargins.ReportingNode = "CH";
allocatedTechnicalMargins.ReportingPeriod = (reportingYear, reportingMonth);
allocatedTechnicalMargins.ColumnSlices = new string[]{"GroupOfContract", "EstimateType"};
allocatedTechnicalMargins.DataFilter = null;//new [] {("GroupOfContract", "RP1.1")};
allocatedTechnicalMargins.Scenario = null;
//allocatedTechnicalMargins.Scenario = "All";
(await allocatedTechnicalMargins.ToReportAsync) with {Height = 700}

# LRC Actuarial


In [0]:
var actuarialLrc = ifrs17.ActuarialLrc;
actuarialLrc.ReportingNode = "CH";
actuarialLrc.ReportingPeriod = (reportingYear, reportingMonth);
actuarialLrc.ColumnSlices = new string[]{"GroupOfContract", "EstimateType"};
actuarialLrc.DataFilter = null;
(await actuarialLrc.ToReportAsync) with {Height = 750}

# Financial Performance


In [0]:
var financialPerformance = ifrs17.FinancialPerformance;
financialPerformance.ReportingNode = "CH";
financialPerformance.ReportingPeriod = (reportingYear, reportingMonth);
financialPerformance.ColumnSlices = new string[]{"GroupOfContract"};
financialPerformance.DataFilter = null;
(await financialPerformance.ToReportAsync) with { Height = 900, GroupDefaultExpanded = 3}